In [3]:
! pip install scikit-learn

In [4]:
import pandas as pd
import sklearn as sk
import numpy as np
import random

### Data Synthesizing

Some issues here that would need to be investigated further, given longer time to produce synthetic data:
- Potential for unrealistic crossover between age and time at current address
- A lot of randomness like for the income field which we could take realistic distributions of, to improve the accuracy of this
- For more realistic results, could have inferred information from features to select where target is 1 and 0. Maybe where poor credit score, unemployed, high current borrowings, criminal record

In [7]:
#Define output path for data and other outputs
output_directory = r"C:\Users\xandi\Downloads\insurance_poc_task"

In [63]:
np.random.seed(42)

n_samples = 10000

# Numerical features
customer_id = np.arange(1, (n_samples+1), 1)
age = np.random.randint(18, 90, size=n_samples)
income = np.random.choice(np.arange(10000, 150001, 1000), size=n_samples).clip(15000, 200000)
credit_score = np.random.normal(650, 70, size=n_samples).clip(300, 850).round().astype(int)
dependents = np.random.poisson(1.5, size=n_samples)
monthly_rent = np.random.randint(600, 3500, size=n_samples)
total_monthly_costs = monthly_rent + (np.random.randint(500, 3000, size=n_samples))
# Categorical features
gender = np.random.choice(['Male', 'Female'], size=n_samples, p=[0.5, 0.5])
marital_status = np.random.choice(['Single', 'Married', 'Divorced', 'Widowed'], size=n_samples)
education = np.random.choice(['High School', 'Bachelors', 'Masters', 'PhD', 'Other'], size=n_samples)
occupation = np.random.choice(['Business Owner', 'Technical', 'Management', 'Sales', 'Services', 'Hospitality', 'Retired', 'Unemployed'], size=n_samples)
time_at_current_address = np.random.choice(['<1', '1-5', '6-10', '11-15', '16-20', '20+'], size=n_samples)
total_current_borrowings = np.random.choice(['0', '1-5000', '5001-20000', '20001-50000', '50001+'], size=n_samples)
employment_duration = np.random.choice(['<1', '1-5', '6-10', '11-15', '16-20', '20+'], size=n_samples)
# Ordinal features
criminal_record = np.random.choice(['No', 'Minor', 'One Conviction', 'Multiple'], size=n_samples)
health_status = np.random.choice(['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'], size=n_samples, p=[0.1, 0.2, 0.3, 0.3, 0.1])
driving_record = np.random.choice(['Bad', 'Average', 'Good', 'Excellent'], size=n_samples)
home_ownership = np.random.choice(['Rent', 'Own', 'Mortgage', 'Other'], size=n_samples)
#Based off location data and classifying crime risk based on publicly available sources
location_risk = np.random.choice(['Low', 'Medium', 'High', 'Other'], size=n_samples)

# Combine into DataFrame
df = pd.DataFrame({
    'customer_id': customer_id,
    'age': age,
    'income': income,
    'credit_score': credit_score,
    'dependents': dependents,
    'monthly_rent': monthly_rent,
    'total_monthly_costs': total_monthly_costs,
    'gender': gender,
    'marital_status': marital_status,
    'education': education,
    'occupation': occupation,
    'time_at_current_address': time_at_current_address,
    'total_current_borrowings': total_current_borrowings,
    'employment_duration': employment_duration,
    'criminal_record': criminal_record,
    'health_status': health_status,
    'driving_record': driving_record,
    'home_ownership': home_ownership,
    'location_risk': location_risk
})

for col in df.columns:
    if col != 'customer_id':
        df.loc[df.sample(frac=0.05).index, col] = np.nan

duplicates = df.sample(frac=0.01)
df = pd.concat([df, duplicates], ignore_index=True)
print(df)

df['claim_status'] = np.random.choice(['True', 'False'], size=len(df), p=[0.9, 0.1])

# Shuffle dataset
df = df.sample(frac=1).reset_index(drop=True)
data_output = output_directory+"/data/synthetic_customer_data.csv"
df.to_csv(output_directory+"/data/synthetic_customer_data.csv", index=False)

       customer_id   age    income  credit_score  dependents  monthly_rent  \
0                1  69.0   94000.0         579.0         0.0        2741.0   
1                2  32.0   72000.0         675.0         2.0        2753.0   
2                3  89.0   81000.0         602.0         2.0        2410.0   
3                4  78.0   15000.0         672.0         0.0        2336.0   
4                5  38.0   37000.0         653.0         2.0        1936.0   
...            ...   ...       ...           ...         ...           ...   
10095         4402  57.0  118000.0         602.0         1.0        3185.0   
10096         2131  40.0   17000.0         714.0         2.0        2886.0   
10097         2969  39.0   15000.0         614.0         1.0        2090.0   
10098          654  72.0   95000.0         630.0         4.0        1013.0   
10099         8009  80.0  118000.0         763.0         0.0        2948.0   

       total_monthly_costs  gender marital_status    education 

##### Data Exploration/Preprocessing
The dataset is now generated and expored, next steps are roughly:
- Initial data exploration to identify data breakdown (any nans, duplications)
- Identifying some correlations between fields, as well as to identify missing and duplicate values

In [64]:
insurance_synth_data = pd.read_csv(data_output)
insurance_synth_data

,customer_id,age,income,credit_score,dependents,monthly_rent,total_monthly_costs,gender,marital_status,education,occupation,time_at_current_address,total_current_borrowings,employment_duration,criminal_record,health_status,driving_record,home_ownership,location_risk,claim_status
0,4002,NaN,32000.0,584.0,2.0,2929.0,4185.0,Male,Married,High School,Management,1-5,0,<1,No,Fair,Average,NaN,Medium,False
1,9443,69.0,112000.0,676.0,3.0,1006.0,2878.0,Female,Single,NaN,Services,<1,50001+,1-5,One Conviction,Good,Good,Rent,High,True
2,8497,59.0,121000.0,651.0,1.0,2276.0,5066.0,Male,Divorced,Masters,Hospitality,11-15,1-5000,11-15,Multiple,Very Good,Bad,Own,High,True
3,1561,58.0,36000.0,576.0,2.0,3248.0,5063.0,Female,Married,High School,Hospitality,<1,20001-50000,11-15,One Conviction,Fair,Good,Rent,High,True
4,1776,67.0,32000.0,543.0,0.0,621.0,1282.0,Male,Divorced,PhD,NaN,<1,20001-50000,11-15,NaN,Very Good,NaN,Rent,Low,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,2127,46.0,93000.0,667.0,3.0,2726.0,5717.0,Female,Divorced,Masters,Technical,20+,20001-50000,16-20,No,Good,Bad,Own,High,True
10096,8692,50.0,57000.0,726.0,0.0,2524.0,3999.0,Male,Widowed,PhD,Technical,<1,0,11-15,Multiple,Fair,Bad,Mortgage,Low,True
10097,6659,39.0,146000.0,616.0,1.0,3234.0,5375.0,Male,Single,PhD,Business Owner,11-15,50001+,20+,Multiple,Poor,Bad,Mortgage,Other,True
10098,1944,49.0,61000.0,739.0,0.0,1271.0,3911.0,Male,Married,High School,Services,16-20,1-5000,1-5,Minor,NaN,Bad,NaN,High,False


In [65]:
print(insurance_synth_data.info())
print(insurance_synth_data.isnull().sum())
insurance_synth_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10100 entries, 0 to 10099
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               10100 non-null  int64  
 1   age                       9597 non-null   float64
 2   income                    9598 non-null   float64
 3   credit_score              9595 non-null   float64
 4   dependents                9598 non-null   float64
 5   monthly_rent              9593 non-null   float64
 6   total_monthly_costs       9590 non-null   float64
 7   gender                    9596 non-null   object 
 8   marital_status            9594 non-null   object 
 9   education                 9597 non-null   object 
 10  occupation                9594 non-null   object 
 11  time_at_current_address   9592 non-null   object 
 12  total_current_borrowings  9597 non-null   object 
 13  employment_duration       9596 non-null   object 
 14  crimin

,customer_id,age,income,credit_score,dependents,monthly_rent,total_monthly_costs
count,10100.000000,9597.000000,9598.000000,9595.000000,9598.000000,9593.000000,9590.000000
mean,4998.902970,53.607690,79640.758491,651.118499,1.499479,2027.750339,3776.261210
std,2887.719095,20.757071,40389.900975,69.862837,1.224489,835.669883,1102.668593
min,1.000000,18.000000,15000.000000,397.000000,0.000000,600.000000,1118.000000
25%,2500.750000,36.000000,45000.000000,603.000000,1.000000,1304.000000,2972.250000
50%,4994.500000,54.000000,79000.000000,651.000000,1.000000,2029.000000,3777.000000
75%,7504.250000,72.000000,115000.000000,699.000000,2.000000,2744.000000,4553.750000
max,10000.000000,89.000000,150000.000000,850.000000,9.000000,3499.000000,6475.000000


##### Looking at the data we can see the majority of fields contain at least some missing values so worth further investigation
- This assumes we have only a small percentage of nulls in each case, if this was a large proportion it maybe worth removal (if considered unimportant) as in this case it would add more noise replacing them than probable benefits

In [ ]:
#Age should be fine to use mean or median value to plug the gaps as the remaining rows are high quality, similarly with income, credit score, dependents, monthly_rent, total_monthly_costs
def mean_replacement(df, list_cols):
    for col in list_cols:
        df[col] = df[col].fillna(round(df[col].mean()))
    return df

columns_mean_replacement = ['age', 'income', 'credit_score', 'dependents', 'monthly_rent', 'total_monthly_costs']
insurance_synth_data = mean_replacement(insurance_synth_data, columns_mean_replacement)

#gender fill evenly
# marital_status, education, occupation, time_at_current_address, total_current_borrowings,  we can fill based on current distribution to maintain consistency
insurance_synth_data




,customer_id,age,income,credit_score,dependents,monthly_rent,total_monthly_costs,gender,marital_status,education,occupation,time_at_current_address,total_current_borrowings,employment_duration,criminal_record,health_status,driving_record,home_ownership,location_risk,claim_status
0,4002,54.0,32000.0,584.0,2.0,2929.0,4185.0,Male,Married,High School,Management,1-5,0,<1,No,Fair,Average,NaN,Medium,False
1,9443,69.0,112000.0,676.0,3.0,1006.0,2878.0,Female,Single,NaN,Services,<1,50001+,1-5,One Conviction,Good,Good,Rent,High,True
2,8497,59.0,121000.0,651.0,1.0,2276.0,5066.0,Male,Divorced,Masters,Hospitality,11-15,1-5000,11-15,Multiple,Very Good,Bad,Own,High,True
3,1561,58.0,36000.0,576.0,2.0,3248.0,5063.0,Female,Married,High School,Hospitality,<1,20001-50000,11-15,One Conviction,Fair,Good,Rent,High,True
4,1776,67.0,32000.0,543.0,0.0,621.0,1282.0,Male,Divorced,PhD,NaN,<1,20001-50000,11-15,NaN,Very Good,NaN,Rent,Low,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,2127,46.0,93000.0,667.0,3.0,2726.0,5717.0,Female,Divorced,Masters,Technical,20+,20001-50000,16-20,No,Good,Bad,Own,High,True
10096,8692,50.0,57000.0,726.0,0.0,2524.0,3999.0,Male,Widowed,PhD,Technical,<1,0,11-15,Multiple,Fair,Bad,Mortgage,Low,True
10097,6659,39.0,146000.0,616.0,1.0,3234.0,5375.0,Male,Single,PhD,Business Owner,11-15,50001+,20+,Multiple,Poor,Bad,Mortgage,Other,True
10098,1944,49.0,61000.0,739.0,0.0,1271.0,3911.0,Male,Married,High School,Services,16-20,1-5000,1-5,Minor,NaN,Bad,NaN,High,False
